# Collaborative Filtering

Collaborative recommendation using Implicit ALS algorithm.   
Data source: Enriched url Data 

In [5]:
import re
import os
import math

import sys
import pandas as pd
import numpy as np
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
import random

from sklearn.preprocessing import MinMaxScaler

import implicit

D:\Codes\Recommendation Engine\Hybrid-Recommender-System\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Preprocessing

## Importing Article Master for Article Lookup

In [6]:
article_master= pd.read_csv(r'./URL_to_Article/URL_ArticleID_cHash_mapping.csv', sep=';')
article_master['reduced_title'] = article_master.apply(lambda row: re.sub('[^A-Za-z0-9]+', '', row.title.lower()), axis = 1)


In [7]:
article_lookup_ID = article_master.loc[:,['article_id', 'title', 'reduced_title']]
article_lookup_ID = article_lookup_ID.drop_duplicates(['article_id', 'reduced_title'], keep='first')
article_lookup_ID = article_lookup_ID.drop(['reduced_title'], axis = 1)
pd.set_option('display.max_colwidth', 200)
article_lookup_ID.head()

,article_id,title
0,12,Bridging the industrial heat divide
1,20,EUSEW 2016 - energy efficiency awards
2,21,How much can energy management actually save?
4,22,All eyes on China’s 13th Five-Year Plan for energy
7,23,EEIP at Turkish Energy Efficiency Week


## Importing Interaction and processing 

In [8]:
raw_interaction = pd.read_csv(r'./Enriched_data/InteractionData.csv')
# raw_interaction = pd.read_csv(r'./Enriched_data/new__report-5670-20190711132352.csv')

raw_interaction = raw_interaction.loc[:,['IP Address', 'Entry Page', 'Page Visits', 'Visit Duration']]

In [9]:
# pd.set_option('max_colwidth', 100)
page_visit_url = raw_interaction["Page Visits"].str.split("-->", expand=True)
page_count = pd.DataFrame(page_visit_url.apply(lambda x: x.count(), axis=1))
page_count = page_count.rename(columns= {0: 'Page Count'})
page_visit_url = raw_interaction['Entry Page'].to_frame().join(page_visit_url)

In [10]:
flag = pd.DataFrame(np.zeros((len(page_visit_url), 1)))
flag = flag.rename(columns= {0: 'Flag'})

## Converting URLs to ArticleID

In [11]:
# Returns ArticleID from for the URL
# Returns 0 if 
#     1. URL starts with www2.
#     2. Article not found.

def urls_to_id(url):
    
    if url is None or (isinstance(url, float) and  math.isnan(url)):
        return 0
    
    #   Remove www. header from the URL
    if url[0:4] == 'www.':
        url = url[4:]
        
    #   If URL starts with www2. return 0.
    if url[0:5] == 'www2.':
        return 0
    
    url_tokens = url.split("/")
    
    #   If URL is not an article return 0
    if len(url_tokens) < 5:
        return 0
    if url_tokens[1]!= 'articles':
        return 0 
    
    
    #   If URL is an article return the article name
    token4 = url_tokens[3]
    token5 = url_tokens[4]
    
    #     If token4 not blank then it must be the article name or the cHash
    if token4 != '':
        
        #   Assuming that the 4th token is the cHash
        search_cHash_rows = article_master.loc[article_master['cHash'] == token4]
        
        #   if no match found in the cHash column
        #   Then token4 is the article name and so token5 is the cHash
        if len(search_cHash_rows) < 1:
            search_cHash_rows = article_master.loc[article_master['cHash'] == token4]
                
        #   Return ArticleID if exact match found with cHash
        if len(search_cHash_rows) == 1: 
            return int(search_cHash_rows['article_id'])
        
        #   Return the fist ArticleID if multiple articles found
        #   This is because cHash and articles have a one-to-one matching
        if len(search_cHash_rows) > 1: 
            return int(search_cHash_rows.loc[search_cHash_rows.index[0], 'article_id'])
        
        #   Return 0 by default if none of the cases match
        return 0
        
    
    #     If token4 blank then token5 must be the article name
    else:
        #   removing spaces and punctuations from the URL article name
        article_name = (re.sub('[A-Za-z0-9]+', '', token5)).lower()
        
        #   Pattern searching on title since the URL might not have the full article name
        #   This also takes care if the situation in which multiple articles have similar titles       
        search_article_name_rows = article_master[article_master['reduced_title'].str.contains(article_name)]
        
        #   Return ArticleID if match found with article_name
        if len(search_article_name_rows) == 1: 
            return int(search_article_name_rows['article_id'])
        
        #   If multiple rows found assume that all rows point to the same article
        #   return the article_id of the first row
        if len(search_article_name_rows) < 1: 
            return int(search_article_name_rows['article_id'][0])
        
        #   Return 0 by default if none of the cases match
        return 0
        

In [12]:
# Flag value notes if the row has any article pages
# If not then flag for that row is 0

cols = list(page_visit_url)
url_article_id_list = []
for i in range(len(page_visit_url)):
    url_article_id_list.append([])
    
    for j in list(page_visit_url):
        article_id = urls_to_id(page_visit_url[j][i])
        
        #  If the PageVisit1 ID is the same as EntryPage ID, then put PageVisit1 as 0
        if j == 0 and url_article_id_list[-1][-1] == article_id:            
            url_article_id_list[-1].append(0)
        
        else:
            url_article_id_list[-1].append(urls_to_id(page_visit_url[j][i]))
            
        if url_article_id_list[-1][-1] != 0:
            flag['Flag'][i] +=1 


# url_article_id_list
page_visit_id = pd.DataFrame(url_article_id_list, columns = cols)

## Creating final dataframe

In [13]:
srt = page_visit_id.iloc[:,0:len(cols)].apply(lambda x: ",".join(x.astype(str)), axis=1)
new_interaction = raw_interaction.loc[:, ~raw_interaction.columns.isin(['Entry Page','Page Visits'])].join(pd.DataFrame(srt))
new_interaction = new_interaction.rename(columns= {0: 'Pages'})
new_interaction = new_interaction.join(flag)
new_interaction = new_interaction.join(page_count)

In [14]:
new_interaction = new_interaction.dropna(subset=['Visit Duration'])
new_interaction = new_interaction[new_interaction['Visit Duration'] > 0]
new_interaction = new_interaction[new_interaction['Flag'] > 0]
new_interaction = new_interaction.reset_index(drop = True)


In [15]:
# cols = ['IPAddress', 'Page', 'Weight']
data_list = []
for i in range(len(new_interaction)):
    ip = (new_interaction.loc[i,['IP Address']]).item()
    duration = float(new_interaction.loc[i,['Visit Duration']])
    count = float(new_interaction.loc[i,['Page Count']])
    stay_count = duration / count
    
    pages = new_interaction.loc[i,['Pages']]
    for i in (pages.item()).split(','):
        if(i != '0'):
            data_list.append([ip, i, stay_count])
            
            
data = pd.DataFrame(data_list, columns = ['IPAddress', 'Page', 'Weight'])

In [16]:
data = data.groupby(['IPAddress', 'Page']).sum().reset_index()
data['Weight'] = (data['Weight'] - data['Weight'].min())/(data['Weight'].max() - data['Weight'].min())
data.head()

,IPAddress,Page,Weight
0,0.253.137.182,100216,0.001891
1,0.87.65.250,100148,0.290241
2,0.87.65.250,100166,0.236517
3,0.87.65.250,100167,0.118241
4,1.110.29.103,100215,0.002100


In [17]:
data['user_id'] = data['IPAddress'].astype("category").cat.codes
data['page_id'] = data['Page'].astype("category").cat.codes

user_lookup = data[['user_id', 'IPAddress']].drop_duplicates()

page_lookup = data[['page_id', 'Page']].drop_duplicates()
page_lookup['Page'] = page_lookup['Page'].astype('int64')
page_lookup = pd.merge(page_lookup, article_lookup_ID, how='inner', left_on = 'Page', right_on = 'article_id')
page_lookup = page_lookup.drop(['Page'], axis = 1)



data = data.drop(['IPAddress', 'Page'], axis = 1)

In [18]:
data = data[['user_id', 'page_id', 'Weight']]
data.head()

,user_id,page_id,Weight
0,0,113,0.001891
1,1,65,0.290241
2,1,82,0.236517
3,1,83,0.118241
4,2,112,0.002100


# Model Creation and Fitting

In [19]:
# The implicit library expects data as a item-user matrix so we
# create two matricies, one for fitting the model (item-user) 
# and one for recommendations (user-item)

sparse_item_user = sparse.csr_matrix((data['Weight'].astype(float), (data['page_id'], data['user_id'])))
sparse_user_item = sparse.csr_matrix((data['Weight'].astype(float), (data['user_id'], data['page_id'])))

In [20]:
# Initialize the als model and fit it using the sparse item-user matrix
os.environ['OPENBLAS_NUM_THREADS'] = '1'
os.environ['MKL_NUM_THREADS'] = '1'
model = implicit.als.AlternatingLeastSquares(factors=20, regularization=0.1, iterations=20)

In [21]:
# Calculate the confidence by multiplying it by our alpha value.
alpha_val = 15
data_conf = (sparse_item_user * alpha_val).astype('double')


In [22]:
#Fit the model
model.fit(data_conf)

100%|█████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 154.85it/s]


# Recommendation

In [23]:
# page_lookup.head()
page_lookup.loc[page_lookup['title'] == 'Utilize all the available energy — Heat recovery']

,page_id,article_id,title
40,109,100211,Utilize all the available energy — Heat recovery


In [24]:
data.head()

,user_id,page_id,Weight
0,0,113,0.001891
1,1,65,0.290241
2,1,82,0.236517
3,1,83,0.118241
4,2,112,0.002100


In [25]:
#---------------------
# FIND SIMILAR ITEMS
#---------------------

# Find the 10 most similar articles to 'Utilize all the available energy — Heat recovery'
item_id = 66
n_similar = 10

# Use implicit to get similar items.
similar = model.similar_items(item_id, n_similar)

# Print the names of our most similar artists
for item in similar:
    idx, score = item
    print(page_lookup.title.loc[page_lookup['page_id'] == idx].iloc[0])

ValueError: too many values to unpack (expected 2)

In [22]:
# Preview of the Users in the database
user_lookup.head()

,user_id,IPAddress
0,0,0.253.137.182
1,1,0.87.65.250
4,2,1.110.29.103
5,3,1.244.13.175
7,4,10.62.16.174


In [23]:
# Preview of the viewing history for user id = 7

pd.merge(page_lookup, data.loc[data['user_id'] == 7], on='page_id')

,page_id,article_id,title,user_id,Weight
0,29,100109,5 steps for a robust energy strategy,7,0.012806
1,75,100159,21st century energy management,7,0.012806
2,89,100175,Using Non-Energy Benefits to Build Better Business Cases,7,0.012806


In [24]:
#------------------------------
# CREATE USER RECOMMENDATIONS
#------------------------------

# Create recommendations for user with id 7
user_id = 7

# Use the implicit recommender.
recommended = model.recommend(user_id, sparse_user_item)

articles = []
scores = []

# Get artist names from ids
for item in recommended:
    idx, score = item
#     articles.append(page_lookup.title.loc[data.page_id.loc[data['page_id'] == idx]].iloc[0])
    articles.append(page_lookup.title.loc[page_lookup['page_id'] == idx].iloc[0])
    scores.append(score)

In [25]:
# Create a dataframe of artist names and scores
recommendations = pd.DataFrame({'articles': articles, 'score': scores})

recommendations

,articles,score
0,Where energy efficiency is really found is in the minds of men and women,0.248680
1,Depleting stupidity,0.048379
2,Strategic: Investing in industrial Energy Efficiency is a major decision,0.038169
3,De-risking energy efficiency projects: a market opportunity for project developers,0.028040
4,Evolution in energy efficiency financing,0.026780
5,Risk minimization for energy efficiency projects,0.023116
6,TOP 3 articles in Energy Efficiency fom the 2nd quarter of 2018,0.010293
7,Energy audit in SMEs could unlock great energy efficiency potential in Europe: a focus on the Italian model.,0.006440
8,The 4th Utility: Compressed Air,0.005980
9,"FAQ - how the ICP process works for industry, district energy and street lighting",0.005424
